# SIIM-ISIC Melanoma Classification


Se define todo el código asociado a la aplicación web para la predicción del cáncer del melanoma. Se divide en dos archivos de python llamados: **app.py** y **auxiliary_functions.py**. Los archivos html se llaman **inicio.html**, **ver_datos.html**, **realizar_prediccion.htlm** y **resultados_prediccion.html**

# 1.- app.py


In [ ]:
from flask import Flask, render_template, request, redirect, url_for, send_from_directory
import os
import json
from PIL import Image
from auxiliary_functions import *

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'

# Rutas
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        id_persona = request.form['id_persona']
        edad = request.form['edad']
        sexo = request.form['sexo']
        localizacion = request.form['localizacion']
        imagen = request.files['imagen']

        # Comprobar si la carpeta para ID de persona ya existe o crearla si no existe
        persona_folder = os.path.join(app.config['UPLOAD_FOLDER'], id_persona)
        if not os.path.exists(persona_folder):
            os.makedirs(persona_folder)
            n_images = 0  # Inicializar n_images a 0 si la carpeta es nueva
        else:
            # Contar el número de imágenes existentes en la carpeta
            existing_images = [filename for filename in os.listdir(persona_folder) if filename.endswith('.jpeg')]
            n_images = len(existing_images)

        # Redimensionar imagen original a 256x256 píxeles y guardarla
        image = Image.open(imagen)
        image_resized = image.resize((256, 256))
        imagen_redimensionada_nombre = f"ISIC_IP_{id_persona[3:]}_redimensionada.jpeg"
        imagen_redimensionada_path = os.path.join(persona_folder, imagen_redimensionada_nombre)
        image_resized.save(imagen_redimensionada_path)

        # Crear y guardar datos.json
        datos = {
            "id_persona": id_persona,
            "sexo": sexo,
            "edad": edad,
            "localizacion": localizacion,
            "imagen_original": imagen_redimensionada_nombre,
            "n_images": n_images + 1
        }

        with open(os.path.join(persona_folder, 'datos.json'), 'w') as datos_file:
            json.dump(datos, datos_file)

        localizacion = "zona_" + localizacion
        # Obtener el diccionario de zonas con valores 0 y 1
        zonas_dict = localizacion_to_dict(localizacion)

        datos2 = {
            "id_persona": id_persona,
            "sexo": sexo,
            "edad": edad
        }

        # Agregar las zonas al diccionario de datos
        datos2.update(zonas_dict)

        # Guardar la imagen en un archivo TFRecord
        tfrec_filename = os.path.join(persona_folder, f"{id_persona}.tfrec")
        image = tf.io.read_file(imagen_redimensionada_path)
        imagen_redimensionada_nombre_bytes = imagen_redimensionada_nombre.encode('utf-8')  # Codifica el nombre en bytes

        # Crear un ejemplo TFRecord
        example = tf.train.Example(features=tf.train.Features(feature={
            'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image.numpy()])),
            'image_name': tf.train.Feature(bytes_list=tf.train.BytesList(value=[imagen_redimensionada_nombre_bytes]))
        }))

        # Escribir el ejemplo en el archivo TFRecord
        with tf.io.TFRecordWriter(tfrec_filename) as writer:
            writer.write(example.SerializeToString())

        # Guardar datos en el archivo CSV
        guardar_datos_en_csv(datos2, csv_file)

        return redirect(url_for('ver_datos'))

    return render_template('inicio.html')

@app.route('/ver_datos', methods=['GET', 'POST'])
def ver_datos():
    personas = []
    id_personas = []
    for id_persona in os.listdir(app.config['UPLOAD_FOLDER']):
        persona_folder = os.path.join(app.config['UPLOAD_FOLDER'], id_persona)
        if os.path.isdir(persona_folder):
            datos_file_path = os.path.join(persona_folder, 'datos.json')
            if os.path.exists(datos_file_path):
                with open(datos_file_path, 'r') as f:
                    datos = json.load(f)
                    personas.append(datos)
                    id_personas.append(datos['id_persona'])

    selected_persona = None
    if request.method == 'POST':
        selected_id = request.form.get('id_persona')
        if selected_id:
            for persona in personas:
                if persona['id_persona'] == selected_id:
                    selected_persona = persona
                    break

    return render_template('ver_datos.html', personas=personas, id_personas=id_personas, selected_persona=selected_persona)

@app.route('/uploads/<path:filename>')
def image(filename):
    return send_from_directory('uploads', filename)

@app.route('/realizar_prediccion', methods=['GET', 'POST'])
def realizar_prediccion():
    usuarios = obtener_usuarios(app.config['UPLOAD_FOLDER'])

    if request.method == 'POST':
        selected_user = request.form['usuario_seleccionado']

        # Leer los datos del archivo "datos.json" para la persona seleccionada
        persona_folder = os.path.join(app.config['UPLOAD_FOLDER'], selected_user)
        datos_json_path = os.path.join(persona_folder, 'datos.json')

        if os.path.exists(datos_json_path):
            with open(datos_json_path, 'r') as json_file:
                datos = json.load(json_file)

                # Crear el DataFrame 'data' para la predicción
                data = cargar_registro_csv(app, selected_user)


                data.rename(columns={'sexo': 'sex', 'edad': 'age_approx'}, inplace=True)

                # Extraer los valores necesarios para la predicción desde el archivo JSON
                sex = datos['sexo']
                age_approx = datos['edad']

                sexo_num = sexo_to_number(sex)

                data['sex'] = sexo_num
                data['age_approx'] = age_approx

                data['age_approx'] = data['age_approx'].astype('int64')

                data['n_images'] = 1

                # Realizar la predicción con el modelo XGBoost

                tfrec_filename = os.path.join(persona_folder, f"{selected_user}.tfrec")

                if os.path.exists(tfrec_filename):
                    # Cargar el conjunto de datos TFRecord
                    dataset = cargar_dataset_tfrec(tfrec_filename)

                    # Crear un generador de lotes a partir del conjunto de datos
                    batch_size = 1  # Puedes ajustar este valor según sea necesario
                    dataset = dataset.batch(batch_size)

                    # Realizar predicciones con el modelo "modelo_pesos"
                    prediccion_imagen = modelo_pesos.predict(dataset)

                print(data)
                probabilidad_melanoma = modelo_xgb.predict_proba(data)[:, 1]
                prob = prediccion_imagen * 0.9 + probabilidad_melanoma * 0.1
                es_melanoma = prob >= 0.13

                path = os.path.join(persona_folder, datos['imagen_original'])

                resultados = {
                    "id_persona": selected_user,
                    "probabilidad_melanoma": prob.tolist(),
                    "diagnostico": es_melanoma.tolist(),
                    "imagen_path": path
                }
                print(resultados)
                return render_template('resultados_prediccion.html', resultados=resultados)

    return render_template('realizar_prediccion.html', usuarios=usuarios)

if __name__ == '__main__':
    # Ruta al archivo de pesos
    ruta_pesos = "pesos_modeloWithFocalLoss-2.h5"
    ruta_xgb = "modelo_xgboost.pkl"

    # Cargar los pesos del modelo
    modelo_pesos = cargar_pesos_modelo(ruta_pesos)
    modelo_xgb = cargar_xgb_modelo(ruta_xgb)

    # Archivo CSV para guardar los datos
    csv_file = os.path.join(app.config['UPLOAD_FOLDER'], 'new_data.csv')

# Verificar si el archivo CSV existe, si no, crearlo
    if not os.path.exists(csv_file):
        with open(csv_file, 'w') as file:
            file.write("id_persona,sexo,edad,zona_head/neck,zona_lower extremity,zona_oral/genital,zona_palms/soles,zona_torso,zona_upper extremity,n_images\n")

    # Inicio de la APP
    app.run(debug=True)


# 2.- auxiliary_functions.py


In [ ]:
import tensorflow as tf
import os
import joblib
from tensorflow.keras import layers
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
import tensorflow.keras.applications.xception as xcep
import pandas as pd
import numpy as np

## ARREGLAR TAMBIÉN LA FORMA DE MOSTRAR DATOS PARA LA PÁGINA WEB Y LUEGO PARA EL MODELO!!!

def cargar_registro_csv(app, id_persona):
    csv_file = os.path.join(app.config['UPLOAD_FOLDER'], 'new_data.csv')

    # Leer el archivo CSV en un DataFrame
    df = pd.read_csv(csv_file)

    # Filtrar el DataFrame para obtener el registro de la persona seleccionada
    registro_persona = df[df['id_persona'] == id_persona].copy()

    # Eliminar la columna 'id_persona' (opcional, dependiendo de tus necesidades)
    registro_persona.drop(columns=['id_persona'], inplace=True)

    return registro_persona

def localizacion_to_dict(localizacion):
    # Definir las zonas de localización
    zonas = ['zona_head/neck', 'zona_lower extremity', 'zona_oral/genital', 'zona_palms/soles', 'zona_torso', 'zona_upper extremity', 'zona_nan']

    # Crear un diccionario con todas las zonas inicialmente establecidas en 0
    localizacion_dict = {zona: 0 for zona in zonas}

    # Verificar si la localización es válida
    if localizacion in localizacion_dict:
        # Establecer la zona seleccionada en 1
        localizacion_dict[localizacion] = 1

    return localizacion_dict

# Función para guardar datos en el archivo CSV
def guardar_datos_en_csv(datos, csv_file):
    with open(csv_file, 'a') as file:
        file.write(f"{datos['id_persona']},{datos['edad']},{datos['sexo']},"
           f"{datos['zona_head/neck']},{datos['zona_lower extremity']},{datos['zona_oral/genital']},"
           f"{datos['zona_palms/soles']},{datos['zona_torso']},{datos['zona_upper extremity']},{datos['zona_nan']}\n")

def sexo_to_number(sexo):

    if sexo == "Masculino":
        sexo = 1
    else:
        sexo = 0

    return sexo

# Obtener la lista de usuarios (carpetas) en la carpeta uploads
def obtener_usuarios(ruta):
    usuarios = []
    for user_folder in os.listdir(ruta):
        user_path = os.path.join(ruta, user_folder)
        if os.path.isdir(user_path):
            usuarios.append(user_folder)
    return usuarios

def cargar_imagen(filename):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (256, 256))
    image = tf.cast(image, tf.float32) / 255.0  # Normalizar la imagen

    return image

def cargar_imagen_tfrec(filename):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
    }

    example = tf.io.parse_single_example(filename, feature_description)
    image = tf.image.decode_jpeg(example['image'], channels=3)
    image = tf.image.resize(image, (256, 256))
    image = tf.cast(image, tf.float32) / 255.0  # Normalizar la imagen

    return image

def cargar_dataset_imagenes(filenames):
    def load_image(filename):
        image = cargar_imagen(filename)
        return image, filename

    dataset = tf.data.Dataset.from_tensor_slices(filenames)
    dataset = dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    return dataset

#Definición de focal loss
def focal_loss(y_true, y_pred):
    gamma = 2.0
    alpha = 0.25
    pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
    return -alpha * tf.pow(1.0 - pt, gamma) * tf.math.log(pt + 1e-7)

# Cargar los pesos del modelo desde el archivo "pesos_modelo.h5"
def cargar_pesos_modelo(ruta_pesos):
    imSize = 256
    modelo = tf.keras.Sequential([
        xcep.Xception(
        input_shape=(imSize, imSize, 3),
        weights='imagenet',
        include_top=False
        ),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    modelo.compile(
        optimizer='adam',
        loss=focal_loss,
        metrics=['accuracy']
    )

    modelo.load_weights(ruta_pesos)

    return modelo

def cargar_xgb_modelo(ruta_xgb):
    modelo = joblib.load(ruta_xgb)
    return modelo


def image_example(image_path, image_name):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (256, 256))
    image = tf.cast(image, tf.uint8)

    feature = {
        "image": tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(image).numpy()])),
        "image_name": tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_name.encode('utf-8')])),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

# Definir una función para cargar el conjunto de datos TFRecord
def cargar_dataset_tfrec(tfrec_filename):
    dataset = tf.data.TFRecordDataset(tfrec_filename)

    def parse_tfrec(example):
        feature_description = {
            'image': tf.io.FixedLenFeature([], tf.string),
            'image_name': tf.io.FixedLenFeature([], tf.string)
        }
        example = tf.io.parse_single_example(example, feature_description)
        image = tf.image.decode_jpeg(example['image'], channels=3)
        image = tf.image.resize(image, (256, 256))
        image = tf.cast(image, tf.float32) / 255.0
        return image, example['image_name']

    dataset = dataset.map(parse_tfrec, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return dataset


# 3.- inicio.html

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Predicción del Cáncer del Melanoma</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <style>
        body {
            background-color: #f8f9fa;
        }
        .container {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }
        .form-container {
            width: 400px;
            padding: 20px;
            border-radius: 10px;
            background-color: white;
            box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);
        }
        .form-container h2 {
            margin-bottom: 20px;
            text-align: center;
        }
        .form-container p {
            margin-bottom: 20px;
            text-align: center;
        }
        .form-container input,
        .form-container select {
            margin-bottom: 10px;
        }
        .form-container button {
            background-color: #007BFF;
            color: #fff;
            border: none;
            border-radius: 5px;
            padding: 10px 20px;
            cursor: pointer;
            width: 100%;
        }
        .form-container a {
            display: block;
            margin-top: 10px;
            text-align: center;
        }
        .error-message {
            color: red;
            font-size: 12px;
            margin-top: 5px;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="form-container">
            <h2>Predicción del Cáncer del Melanoma</h2>
            <p>Ingresa los datos necesarios para la predicción del cáncer del melanoma.</p>
            <form method="POST" enctype="multipart/form-data" id="melanoma-form">
                <div class="form-group">
                    <input type="text" class="form-control" name="id_persona" placeholder="ID de la persona (ejemplo: IP_X con X número)" required>
                    <div class="error-message" id="id-error"></div>
                </div>
                <div class="form-group">
                    <input type="number" class="form-control" name="edad" placeholder="Edad (0-99)" required>
                    <div class="error-message" id="edad-error"></div>
                </div>
                <div class="form-group">
                    <select class="form-control" name="sexo" required>
                        <option value="" disabled selected>Seleccione el sexo</option>
                        <option value="Masculino">Masculino</option>
                        <option value="Femenino">Femenino</option>
                    </select>
                    <div class="error-message" id="sexo-error"></div>
                </div>
                <div class="form-group">
                    <select class="form-control" name="localizacion" required>
                        <option value="" disabled selected>Seleccione la localización</option>
                        <option value="lower extremity">Extremidad inferior</option>
                        <option value="cabeza/cuello">Cabeza/Cuello</option>
                        <option value="palms/soles">Palmas/Plantas</option>
                        <option value="torso">Torso</option>
                        <option value="zona_oral/genital">Oral/Genital</option>
                        <option value="upper extremity">Extremidad superior</option>
                    </select>
                    <div class="error-message" id="localizacion-error"></div>
                </div>
                <div class="form-group">
                    <input type="file" class="form-control-file" name="imagen" required>
                    <div class="error-message" id="imagen-error"></div>
                </div>
                <button type="submit" class="btn btn-primary">Enviar</button>
                <a href="{{ url_for('ver_datos') }}" class="btn btn-primary">Ver Datos</a>
            </form>
        </div>
    </div>
    <script>
        const form = document.getElementById('melanoma-form');
        const idInput = document.getElementsByName('id_persona')[0];
        const edadInput = document.getElementsByName('edad')[0];

        form.addEventListener('submit', function (event) {
            let valid = true;

            // Validación del ID
            if (!/^IP_\d+$/.test(idInput.value)) {
                document.getElementById('id-error').textContent = 'ID de la persona no válido. Debe comenzar por IP_X donde X es un número.';
                valid = false;
            }

            // Validación de la Edad
            const edad = parseInt(edadInput.value);
            if (isNaN(edad) || edad < 0 || edad > 99) {
                document.getElementById('edad-error').textContent = 'Edad no válida. Debe estar entre 0 y 99.';
                valid = false;
            }

            if (!valid) {
                event.preventDefault();
            }
        });
    </script>
</body>
</html>


# 4.- ver_datos.html

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Datos Ingresados</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <style>
        body {
            background-color: #f8f9fa;
        }
        .container {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }
        .form-container {
            width: 400px;
            padding: 20px;
            border-radius: 10px;
            background-color: white;
            box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);
        }
        .form-container h2 {
            margin-bottom: 20px;
            text-align: center;
        }
        .form-container p {
            margin-bottom: 10px;
            text-align: center;
        }
        .form-group p {
            margin-bottom: 5px;
            font-weight: bold;
        }
        .form-container a {
            display: block;
            margin-top: 20px;
        }
        .centered-img {
            display: block;
            margin: 0 auto;
            max-width: 100%;
            height: auto;
        }
        .centered-content {
            text-align: center;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="form-container">
            <h2>Datos Ingresados</h2>
            <p>Aquí puedes ver los datos que has ingresado:</p>
            {% if selected_persona %}
            <div class="form-group centered-content">
                <p><strong>ID de Persona:</strong> {{ selected_persona.id_persona }}</p>
                <p><strong>Edad:</strong> {{ selected_persona.edad }}</p>
                <p><strong>Sexo:</strong> {{ selected_persona.sexo }}</p>
                <p><strong>Localización del Melanoma:</strong> {{ selected_persona.localizacion }}</p>
                <div class="form-group text-center">
                    <p><strong>Imagen Original:</strong></p>
                    <img class="centered-img" src="{{ url_for('image', filename=selected_persona.id_persona + '/' + selected_persona.imagen_original) }}" alt="Imagen Original">
                </div>
            </div>
            {% else %}
            <p>No hay datos introducidos hasta el momento.</p>
            {% endif %}
            <form action="{{ url_for('ver_datos') }}" method="POST">
                <div class="form-group">
                    <select class="form-control" name="id_persona">
                        <option value="" disabled selected>Seleccione ID de Persona</option>
                        {% for id in id_personas %}
                        <option value="{{ id }}">{{ id }}</option>
                        {% endfor %}
                    </select>
                </div>
                <button type="submit" class="btn btn-primary">Ver Datos Seleccionados</button>
                <a href="{{ url_for('realizar_prediccion') }}" class="btn btn-primary mt-3">Realizar Predicción</a>
            </form>
            <a href="/" class="btn btn-primary mt-3">Volver a Ingresar Datos</a>
        </div>
    </div>
</body>
</html>

#5.- realizar_prediccion.html

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Realizar Predicción</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <style>
        body {
            background-color: #f8f9fa;
        }
        .container {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }
        .form-container {
            width: 400px;
            padding: 20px;
            border-radius: 10px;
            background-color: white;
            box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);
        }
        .form-container h2 {
            margin-bottom: 20px;
            text-align: center;
        }
        .form-container p {
            margin-bottom: 20px;
            text-align: center;
        }
        .form-group p {
            margin-bottom: 5px;
            font-weight: bold;
        }
        .form-container a {
            display: block;
            margin-top: 20px;
        }
        .centered-img {
            display: block;
            margin: 0 auto;
            max-width: 100%;
            height: auto;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="form-container">
            <h2>Realizar Predicción</h2>
            <form action="/realizar_prediccion" method="POST">
                <div class="form-group">
                    <label for="usuario_seleccionado">Seleccione Usuario:</label>
                    <select class="form-control" name="usuario_seleccionado">
                        {% for usuario in usuarios %}
                        <option value="{{ usuario }}">{{ usuario }}</option>
                        {% endfor %}
                    </select>
                </div>
                <button type="submit" class="btn btn-primary">Realizar Predicción</button>
            </form>
            <a href="/" class="btn btn-primary mt-3">Volver a Ingresar Datos</a>
            <a href="/ver_datos" class="btn btn-primary mt-3">Ver Datos Ingresados</a>
        </div>
    </div>
</body>
</html>


# 6.- resultados_prediccion.html

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Resultados de Predicción</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <style>
        body {
            background-color: #f8f9fa;
        }
        .container {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }
        .form-container {
            width: 400px;
            padding: 20px;
            border-radius: 10px;
            background-color: white;
            box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);
        }
        .form-container h2 {
            margin-bottom: 20px;
            text-align: center;
        }
        .form-container p {
            margin-bottom: 20px;
            text-align: center;
        }
        .form-group p {
            margin-bottom: 5px;
            font-weight: bold;
        }
        .form-container a {
            display: block;
            margin-top: 20px;
        }
        .centered-img {
            display: block;
            margin: 10px auto; /* Aumenta el margen inferior para separar la imagen del texto "Imagen" */
            max-width: 100%;
            height: auto;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="form-container">
            <h2>Resultados de Predicción</h2>
            <ul class="list-group">
                <li class="list-group-item">
                    <strong>ID de Persona:</strong> {{ resultados.id_persona }}<br>
                    <strong>Probabilidad de Melanoma:</strong> {{ resultados.probabilidad_melanoma }}<br>
                    <strong>Diagnóstico:</strong> {% if resultados.diagnostico %}Melanoma{% else %}No Melanoma{% endif %}<br>
                    <br>
                    <!-- Agrega la imagen después de la etiqueta "Imagen" con un margen inferior -->
                    <strong>Imagen:</strong>
                    <img src="{{ resultados.imagen_path }}" class="centered-img" alt="Imagen">
                </li>
            </ul>
            <a href="/" class="btn btn-primary mt-3">Volver a Ingresar Datos</a>
            <a href="/ver_datos" class="btn btn-primary mt-3">Ver Datos Ingresados</a>
            <a href="/realizar_prediccion" class="btn btn-primary mt-3">Volver a Realizar Predicción</a>
        </div>
    </div>
</body>
</html>
